## Necessary imports

In [1]:
import gzip
import json
import shutil
import pandas as pd
from datetime import datetime

## Create Dataframes for Each Table

In [2]:
receipts = pd.read_json('receipts.json', lines=True)
brands = pd.read_json('brands.json', lines=True)

# the JSON file for users wasn't cooperating 
with gzip.open('users.json.gz', 'rb') as file_in:
          with open('users.json.gz'.replace('.gz',''), 'wb') as file_out:
            shutil.copyfileobj(file_in, file_out)

users = pd.read_json('users.json', lines=True)

## Parse data for ID column in Brands Table

In [3]:
# create empty dataframe
cpgDF = pd.DataFrame(columns=['$ref'])

for index,row in brands.iterrows():
    if isinstance(row['_id'], dict) and len(row['_id']) == 1:
        brands.loc[index, "_id"] = row['_id']['$oid']
    if isinstance(row['cpg'], dict) and len(row['cpg']) != 0:
        cpgDF = cpgDF.append(row['cpg'], ignore_index=True)
        

# parse cpgDF further
for index,row in cpgDF.iterrows():
    if isinstance(row['$id'], dict) and len(row['$id']) == 1:
        cpgDF.loc[index, "$id"] = row['$id']['$oid']

# combine cpgDF and Brands table together using an outer join
brands = brands.join(cpgDF,how='outer', lsuffix='left', rsuffix='right')

# drop cpg column
brands.drop('cpg', axis=1, inplace=True)

## Parse data for nested JSONs in Users Table

In [4]:
for index,row in users.iterrows():
    if isinstance(row['_id'], dict) and len(row['_id']) == 1:
        users.loc[index, "_id"] = row['_id']['$oid']
    if isinstance(row['createdDate'], dict) and len(row['createdDate']) == 1:
        users.loc[index, "createdDate"] = row['createdDate']['$date']
    if isinstance(row['lastLogin'], dict) and len(row['lastLogin']) == 1:
        users.loc[index, "lastLogin"] = row['lastLogin']['$date']


## Parse Data for Nested JSONs in Receipts Table
## Create a new dataframe for rewardsReceiptItemList and join it with Receipts table

In [5]:
# create empty dataframe
rewardsReceiptDF = pd.DataFrame(columns=['barcode'])

# iterate through receipts dataframe and remove nested jsons
for index,row in receipts.iterrows():
    if isinstance(row['_id'], dict) and len(row['_id']) == 1:
        receipts.loc[index, "_id"] = row['_id']['$oid']
    if isinstance(row['createDate'], dict) and len(row['createDate']) == 1:
        receipts.loc[index, "createDate"] = row['createDate']['$date']
    if isinstance(row['dateScanned'], dict) and len(row['dateScanned']) == 1:
        receipts.loc[index, "dateScanned"] = row['dateScanned']['$date']
    if isinstance(row['finishedDate'], dict) and len(row['finishedDate']) == 1:
        receipts.loc[index, "finishedDate"] = row['finishedDate']['$date']
    if isinstance(row['modifyDate'], dict) and len(row['modifyDate']) == 1:
        receipts.loc[index, "modifyDate"] = row['modifyDate']['$date']
    if isinstance(row['pointsAwardedDate'], dict) and len(row['pointsAwardedDate']) == 1:
        receipts.loc[index, "pointsAwardedDate"] = row['pointsAwardedDate']['$date']
    if isinstance(row['purchaseDate'], dict) and len(row['purchaseDate']) == 1:
        receipts.loc[index, "purchaseDate"] = row['purchaseDate']['$date']
    if isinstance(row['rewardsReceiptItemList'], list) and len(row['rewardsReceiptItemList']) != 0:
        # create a temp dictionary 
        temp_dict = {k:v for e in row['rewardsReceiptItemList'] for k,v in e.items()}
        rewardsReceiptDF = rewardsReceiptDF.append(temp_dict, ignore_index=True)

# combine the dataframes using an outer join
receipts = receipts.join(rewardsReceiptDF,how='outer', lsuffix='left', rsuffix='right')

# drop rewardsReceiptItemList column
receipts.drop('rewardsReceiptItemList', axis=1, inplace=True)

## Transform the dates to be readable

In [6]:
def transform_date(timestamp):
    try:
        # timestamp is in milliseconds, diving by 1000 will convert to seconds
        # I'm using local time, but can convert to UTC as well 
        return datetime.fromtimestamp(timestamp/1000)
    except:
        return None

## Update Users and Receipts DataFrames

In [7]:
users['createdDate'] = users['createdDate'].apply(lambda time: transform_date(time))
users['lastLogin'] = users['lastLogin'].apply(lambda time: transform_date(time))

for col in receipts.columns:
    if 'date' in col.lower():
        receipts[col] = receipts[col].apply(lambda time: transform_date(time))

## Create Table in MySQL 

In [16]:
from sqlalchemy import create_engine

with open('/Users/cynthia/Desktop/password.txt','r') as file:
    password = file.readline()

hostname='127.0.0.1'
dbname='sys'
uname='root'
pwd=password

engine = create_engine(f"mysql+pymysql://{uname}:{pwd}@{hostname}/{dbname}")

In [9]:
receipts.to_sql('Receipts', engine, if_exists='replace', index=False)
users.to_sql('Users', engine, if_exists='replace', index=False)
brands.to_sql('Brands', engine, if_exists='replace', index=False)

## Execute Queries

In [10]:
import mysql.connector

config = {
  'user':uname,
  'password':pwd,
  'host':hostname,
  'database':dbname,
  'raise_on_warnings': True
}

try:
    cnx = mysql.connector.connect(**config)
except Error as e:
    print(e)

## When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

I'll assume that Finished and Accepted are interchangeable. 

Average spend from receipts with 'rewardsReceiptStatus' of accepted is greater than receipts with 'rewardsReceiptStatus' of rejected.

In [11]:
query1 = """
SELECT r.rewardsReceiptStatus, avg(r.totalSpent)
FROM receipts r
GROUP BY r.rewardsReceiptStatus;
"""

with cnx.cursor() as cursor:
    cursor.execute(query1)
    for item in cursor.fetchall():
        print(item)

('FINISHED', 80.85430501930502)
('REJECTED', 23.326056338028184)
('FLAGGED', 180.4517391304348)
('SUBMITTED', None)
('PENDING', 28.03244897959184)


## When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Assuming accepted is synonymous with finished, receipts with 'rewardsReceiptStatus’ of accepted had more purchases than receipts with 'rewardsReceiptStatus’ of rejected.

In [12]:
query2 = """
SELECT r.rewardsReceiptStatus, sum(r.purchasedItemCount)
FROM receipts r
GROUP BY r.rewardsReceiptStatus;
"""

with cnx.cursor() as cursor:
    cursor.execute(query2)
    for item in cursor.fetchall():
        print(item)

('FINISHED', 8184.0)
('REJECTED', 173.0)
('FLAGGED', 1014.0)
('SUBMITTED', None)
('PENDING', None)


### Questions 5 and 6 couldn't be queried, 6 months ago from the day I'm writing this script was: April 14th however, the most recently created user was created on Feb 12th (a little over 8 months ago)

In [13]:
query3 = """
SELECT createdDate from users
ORDER BY createdDate desc
LIMIT 10;
"""

with cnx.cursor() as cursor:
    cursor.execute(query3)
    for item in cursor.fetchall():
        print(item)

(datetime.datetime(2021, 2, 12, 8, 11, 6),)
(datetime.datetime(2021, 2, 12, 8, 10, 49),)
(datetime.datetime(2021, 2, 11, 10, 17, 54),)
(datetime.datetime(2021, 2, 11, 10, 17, 50),)
(datetime.datetime(2021, 2, 11, 10, 17, 8),)
(datetime.datetime(2021, 2, 11, 8, 1, 17),)
(datetime.datetime(2021, 2, 11, 8, 0, 47),)
(datetime.datetime(2021, 2, 10, 13, 53, 2),)
(datetime.datetime(2021, 2, 9, 8, 17, 53),)
(datetime.datetime(2021, 2, 9, 8, 17, 53),)


In [14]:
# close connection
cnx.close()